In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.cloud import bigquery
client = bigquery.Client()

In [5]:
sql = """
SELECT * FROM `mkt2.pricing_data2`
"""
df = client.query(sql).to_dataframe()

In [6]:
df.head()

,prod_id,store_id,week_num,log_demand,price_div_demand,sub_price,com_prices,log_sv,promo_perc
0,20300320,1027,33,4.727388,0.023403,2.893151,3.897396,10.68915,0.000307
1,20300320,1027,31,4.060443,0.047058,2.970268,3.838735,10.68915,0.000231
2,20300320,1027,22,4.553877,0.028555,2.916516,3.815799,10.68915,0.000246
3,20300320,1027,10,4.465908,0.024680,3.027881,3.929881,10.68915,0.023592
4,20300320,1027,3,4.820282,0.021815,3.081854,4.022153,10.68915,0.000253


In [7]:
# Check missing values
df.isna().sum()

prod_id             0
store_id            0
week_num            0
log_demand          0
price_div_demand    0
sub_price           0
com_prices          0
log_sv              0
promo_perc          0
dtype: int64

In [8]:
df['id'] = df['prod_id'].astype(str) + '_' + df['store_id'].astype(str)
df.head()

,prod_id,store_id,week_num,log_demand,price_div_demand,sub_price,com_prices,log_sv,promo_perc,id
0,20300320,1027,33,4.727388,0.023403,2.893151,3.897396,10.68915,0.000307,20300320_1027
1,20300320,1027,31,4.060443,0.047058,2.970268,3.838735,10.68915,0.000231,20300320_1027
2,20300320,1027,22,4.553877,0.028555,2.916516,3.815799,10.68915,0.000246,20300320_1027
3,20300320,1027,10,4.465908,0.024680,3.027881,3.929881,10.68915,0.023592,20300320_1027
4,20300320,1027,3,4.820282,0.021815,3.081854,4.022153,10.68915,0.000253,20300320_1027


In [9]:
from statsmodels.formula.api import ols
ids = df['id'].unique()

In [10]:
# Fit regression models for each product ID
models = {}
for id in ids:
    product_data = df[df['id'] == id]
    model = ols("log_demand ~ price_div_demand + sub_price + com_prices + log_sv + promo_perc", data=product_data).fit()
    models[id] = model

In [11]:
from scipy.optimize import root_scalar
def elasticity(p, a, b, c, d, e, f, sub, com, sv, promo):
    return b * p / (1 + np.exp(-(a + b * p + c * sub + d * com + e * sv + f * promo)))
def elasticity_diff(p, a, b, c, d, e, f, sub, com, sv, promo):
    exponent = - (a + b * p + c * sub + d * com + e * sv + f * promo)
    exponent_clipped = np.clip(exponent, -np.inf, 700)  # Clip the exponent to avoid overflow
    return b * p / (1 + np.exp(exponent_clipped)) + 1

In [13]:
optimal_p_values = {}
for id, model in models.items():
    product_data = df[df['id'] == id]
    
    # Get the average values for the independent variables
    avg_sub = product_data['sub_price'].mean()
    avg_com = product_data['com_prices'].mean()
    avg_sv = product_data['log_sv'].mean()
    avg_promo = product_data['promo_perc'].mean()
    
    # Get the regression coefficients
    a, b, c, d, e, f = model.params['Intercept'], model.params['price_div_demand'], model.params['sub_price'], model.params['com_prices'], model.params['log_sv'], model.params['promo_perc']
    
    # Find the optimal p using the root_scalar function
    res = root_scalar(elasticity_diff, args=(a, b, c, d, e, f, avg_sub, avg_com, avg_sv, avg_promo), method='secant', x0=0.01, x1=0.1)
    optimal_p_values[id] = res.root

In [14]:
optimal_p_values

{'20300320_1027': 0.03623123703514246,
 '20083454001_1027': 0.04763778320449965,
 '20182857_1027': 0.06592709742669038,
 '20310940001_1027': 2.3517287190987384,
 '20133884001_1027': 0.03922919237568135,
 '20026161001_1027': 0.30306318710728763,
 '20168304001_1027': 0.08185297050558123,
 '20425893001_1027': 0.08604112456837375,
 '20426596001_1027': 0.043433956957683734,
 '20970982_1027': 0.4197753112607278,
 '20300320_1028': 0.11479983342274637,
 '20426596001_1028': 0.037310710300583194,
 '20300320_1029': 0.04743657848408355,
 '20374621_1029': 0.04755403136267116,
 '20183964001_1029': 0.05669860481828788,
 '20168304001_1029': 0.2500854957994504,
 '20425893001_1029': 0.1199986095304564,
 '20426596001_1029': 0.03950311230481983,
 '20314039_1029': 0.06581834611184409,
 '21097782001_1029': 0.0663526432414257,
 '20300320_1032': 0.09026938428464448,
 '20133884001_1032': 0.06384109965246089,
 '20183964001_1032': 0.2299102356260253,
 '20026161001_1032': 0.07309533476505967,
 '20426596001_1032':

In [15]:
# Create an empty list to store the results
optimal_prices = []
optimal_demands = []

for id in ids:
    model = models[id]
    a, b, c, d, e, f = model.params['Intercept'], model.params['price_div_demand'], model.params['sub_price'], model.params['com_prices'], model.params['log_sv'], model.params['promo_perc']
    product_data = df[df['id'] == id]
    avg_demand = np.exp(product_data['log_demand'].mean())
    avg_sub_price = product_data['sub_price'].mean()
    avg_com_prices = product_data['com_prices'].mean()
    avg_log_sv = product_data['log_sv'].mean()
    avg_promo_perc = product_data['promo_perc'].mean()

    optimal_p = optimal_p_values[id]
    normal_price = optimal_p * avg_demand
    optimal_demand_logit = a + b*optimal_p + c*avg_sub_price + d*avg_com_prices + e*avg_log_sv + f*avg_promo_perc
    optimal_demand = np.exp(optimal_demand_logit)

    optimal_prices.append(normal_price)
    optimal_demands.append(optimal_demand)

In [16]:
optimal_df = pd.DataFrame({'id': ids, 'optimal_price': optimal_prices, 'optimal_demand': optimal_demands})

In [17]:
optimal_df[['prod_id', 'store_id']] = optimal_df['id'].str.split('_', expand=True)

In [18]:
optimal_df = optimal_df[['prod_id', 'store_id', 'optimal_price', 'optimal_demand']]

In [22]:
optimal_df['optimal_revenue'] = optimal_df['optimal_price'] * optimal_df['optimal_demand']

In [23]:
optimal_df

,prod_id,store_id,optimal_price,optimal_demand,optimal_revenue
0,20300320,1027,3.686462,79.062381,291.460501
1,20083454001,1027,7.098026,113.967808,808.946469
2,20182857,1027,4.663687,54.292083,253.201287
3,20310940001,1027,90.390415,15.830767,1430.949616
4,20133884001,1027,4.837629,93.792889,453.735185
...,...,...,...,...,...
163,20026161001,1021,11.958891,59.273185,708.841536
164,20426596001,1021,7.712224,143.594135,1107.430116
165,20300320,1022,3.215830,88.289609,283.924387
166,20182857,1022,4.665928,81.608584,380.779742


In [24]:
optimal_df.to_csv("gs://rochjia/optimal_store.csv")